# P3Alpha

### Import libraries

In [ ]:
import pandas as pd
import numpy as np

### Read data

In [ ]:
# base path for csv files
base_path = "data"
interactions_df_path = base_path + "/interactions_and_impressions.csv"
items_length_df_path = base_path + "/data_ICM_length.csv"
items_type_df_path = base_path + "/data_ICM_type.csv"
users_df_path = base_path + "/data_target_users_test.csv"

In [ ]:
dtype={0:int, 1:int, 2:str, 3:int}
interactions_df = pd.read_csv(filepath_or_buffer=interactions_df_path, dtype=dtype)
items_length_pf = pd.read_csv(filepath_or_buffer=items_length_df_path)
items_types_df = pd.read_csv(filepath_or_buffer=items_type_df_path)
users_df = pd.read_csv(filepath_or_buffer=users_df_path)